In [1]:
#PURPOSE: Check CPU Capability
import os, sys, time, glob, io, pprint, re, shutil
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import random
import json

import matplotlib.pyplot as plt
import h5py
from datetime import datetime

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
#Plot Data Function
def plot_data_heatmap(data, ID = 'N/A', water_pert='N/A'):
  datap = np.transpose(data) 
  #------------------------------------------------------------------------------------------------------------------------
  #Plot data
  fig1 = plt.figure(figsize=(8,8))
  plt.title('Model ' + str(ID) +  '   (' + str(water_pert) + '% pertubation)', fontsize = 15, pad =12)
  plt.tick_params(axis='both', labelsize=12)
  im1 = plt.imshow((datap), aspect='auto', cmap='RdBu', extent = [0, 100, 5896, 1250])
  plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
  plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)
  cbar = fig1.colorbar(im1)
  cbar.set_label('Waveform amplitude', fontsize = 15, labelpad=25, rotation=270)
  plt.clim(-5,5)
  #fname = 'MIT Images/SYMAE/Variable Seafloor/Shot_gather_0001.00.png'
  #plt.savefig(fname)
  plt.show()
  return None 

In [4]:
#Gather Data and Decide on Cropping the Shot Gather Dataset. 
#Information for cropping: 
##8.8s: time range: 8.845657725001187, n timesteps: 14262dt: 0.000620269106303989, sampling = 19
##8.846/750 = 0.0118 = dt after sampling
##when water velocity is the fastest, and seafloor is the highest, t for direct arrival = d/v = 2000m/1600ms-1 = 1.25s
#1.25/0.0118 = 105 so fir cropping, we should be able to remove the first 105 time points from the 8.8s dataset.
#------------------------------------------------------------------------------------------------------------------------
#Set flags
crop = True
crop_start = 95 #95 for constant seafloor 8.8s 25.50 vel dataset, 105 for constant seafloor 8.8s dataset, 90 for constant seafloor dataset
vel_range = '6.6'
#------------------------------------------------------------------------------------------------------------------------
#Retrieve and Crop Data (w stands for wide, n stands for narrow)
fnameval_w = "ConstantSeafloor8.8s" + vel_range + "vel/valconstantseafloor8.8s" + vel_range + "vel_seismic"
fnameval_n = "ConstantSeafloor8.8s/valconstantseafloor8.8s_seismic"

fnameval_ref_w = "ConstantSeafloor8.8s" + vel_range + "vel/valconstantseafloor8.8s" + vel_range + "vel_seismic_refs"
fnameval_ref_n = "ConstantSeafloor8.8s/valconstantseafloor8.8s_seismic_refs"

datval_w = h5py.File(fnameval_w+".h5", 'r') #val dataset
datval_ref_w=h5py.File(fnameval_ref_w+".h5", 'r') #val dataset
datval_n= h5py.File(fnameval_n+".h5", 'r') #val dataset
datval_ref_n=h5py.File(fnameval_ref_n+".h5", 'r') #val dataset

if crop == False: 
    Xo_val_w=np.array(datval_w["data"])[:,:,:,1:,:]
    Xo_val_ref_w = np.array(datval_ref_w["data"])[:,:,:,1:,:]
    Xo_val_n=np.array(datval_n["data"])[:,:,:,1:,:]
    Xo_val_ref_n = np.array(datval_ref_n["data"])[:,:,:,1:,:]

if crop == True: 
    Xo_val_w=np.array(datval_w["data"])[:,:,:,crop_start:655,:]
    Xo_val_ref_w = np.array(datval_ref_w["data"])[:,:,:,crop_start:655,:]
    Xo_val_n=np.array(datval_n["data"])[:,:,:,crop_start:655,:]
    Xo_val_ref_n = np.array(datval_ref_n["data"])[:,:,:,crop_start:655,:]

#get information for each subsurface model and instance - from the perturbed category.   
nsamp, ntau, n1, n2, nfilt=(700, 10, 100, 560, 1)
nval = 100
with open(str(fnameval_w+'.json')) as f:
   labels_val_w = json.load(f)
labels_val_w=[labels_val_w[i] for i in range(0,nval)]

with open(str(fnameval_n+'.json')) as f:
   labels_val_n = json.load(f)
labels_val_n=[labels_val_n[i] for i in range(0,nval)]


OSError: Unable to open file (unable to open file: name = '/content/drive/MyDrive/seismicTL_circular1_Brindha/ConstantSeafloor8.8s6.6vel/valconstantseafloor8.8s6.6vel_seismic.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [5]:
#Get datasets for splicing and redatuming   
#We want a perturbated gather that has a significant deviation from the pertured
#------------------------------------------------------------------------------------------------------------------------
#Pick samples and instances for validation and reference gather
val_sample = 0 #any number between 0 and 99 excluding the reference samples. (lets focus on 0,2,3,3) for models (901, 902, 903, 904)
val_ins_plot = 7 #any number between 0 and 9
ref_sample_1 = 86
ref_sample_2 = 78
ref_sample_3 = 98
#ref_sample_plot = random.randint(0, 20)
#------------------------------------------------------------------------------------------------------------------------
#Retrieve data
#Take two random references and look at shot gathers to see the differences. Take reference gathers from wide dataset, though they should be identical to references from narrow dataset. 
reference_gather_1 = Xo_val_ref_w[ref_sample_1:ref_sample_1+1,:,:,:,:]
reference_gather_2 = Xo_val_ref_w[ref_sample_2:ref_sample_2+1,:,:,:,:]
reference_gather_3 = Xo_val_ref_w[ref_sample_3:ref_sample_3+1,:,:,:,:]

# #ten_reference_gathers = Xo_val_ref[ref_sample:ref_sample+10,:,:,:,:]
# #Take validation gather sample and corresponding observed instances and reference instances. 
# perturbed_val_gather_1 = Xo_val[val_sample:val_sample+1,:,:,:,:] #0:1
# perturbed_val_gather_2 = Xo_val[val_sample+1:val_sample+2,:,:,:,:]
# perturbed_val_gather_3 = Xo_val[val_sample+2:val_sample+3,:,:,:,:] #0:1
# perturbed_val_gather_4 = Xo_val[val_sample+3:val_sample+4,:,:,:,:]
# reference_val_gather_1 = Xo_val_ref[val_sample:val_sample+1,:,:,:,:] #0:1
# reference_val_gather_2 = Xo_val_ref[val_sample+1:val_sample+2,:,:,:,:] #0:1
# reference_val_gather_3 = Xo_val_ref[val_sample+2:val_sample+3,:,:,:,:]
# reference_val_gather_4 = Xo_val_ref[val_sample+3:val_sample+4,:,:,:,:]

# val_reflector_depth = round(labels_val[val_sample][str(1)]['reflector_depth'], 3)
# val_toplayer_vel = labels_val[val_sample][str(1)]['subsurface_velocities'][0]
# val_ID = labels_val[val_sample][str(val_ins_plot)]['ID']
# val_water_pert = round(labels_val[val_sample][str(val_ins_plot)]['water_pertubation'], 3)

# ref_reflector_depth_1 = round(labels_val[ref_sample_1][str(1)]['reflector_depth'], 3)
# ref_toplayer_vel_1 = labels_val[ref_sample_1][str(1)]['subsurface_velocities'][0]
# ref_ID_1 = labels_val[ref_sample_1][str(1)]['ID']
# ref_ID_1 = ref_ID_1[:5] + '00' #correct for instance number as it is a zero. 

# ref_reflector_depth_2 = round(labels_val[ref_sample_2][str(1)]['reflector_depth'], 3)
# ref_toplayer_vel_2 = labels_val[ref_sample_2][str(1)]['subsurface_velocities'][0]
# ref_ID_2 = labels_val[ref_sample_2][str(1)]['ID']
# ref_ID_2 = ref_ID_2[:5] + '00' #correct for instance number as it is a zero.

# ref_reflector_depth_3 = round(labels_val[ref_sample_3][str(1)]['reflector_depth'], 3)
# ref_toplayer_vel_3 = labels_val[ref_sample_3][str(1)]['subsurface_velocities'][0]
# ref_ID_3 = labels_val[ref_sample_3][str(1)]['ID']
# ref_ID_3 = ref_ID_3[:5] + '00' #correct for instance number as it is a zero.

# ref_water_pert = 0.00

# print("perturbation value = ", str(val_water_pert))
# print("shape of a_reference_gather",np.shape(reference_gather_1))
# #print("shape of ten_reference_gathers",np.shape(ten_reference_gathers))
# print("shape of perturbed_val_gather",np.shape(perturbed_val_gather_1))
# print("shape of reference_val_gather",np.shape(reference_val_gather_1))

# #------------------------------------------------------------------------------------------------------------------------
# #View Validation Sample and View Reference Model
# #View Validation Sample
# valDataPlot = reference_val_gather_1[0,0,:,:,0]
# refDataPlot1= reference_gather_1[0,0,:,:,0]
# refDataPlot2= reference_gather_2[0,0,:,:,0]
# refDataPlot3= reference_gather_3[0,0,:,:,0]

# plot_data_heatmap(valDataPlot, ID = val_ID, water_pert=val_water_pert)
# print ("reflector depth: ", val_reflector_depth)
# print ("top layer velocity: ", val_toplayer_vel)
# print ("primary reflector time = 1.5+ (2621/2163)*2 = 3.92s")


# plot_data_heatmap(refDataPlot1, ID = ref_ID_1, water_pert=ref_water_pert)
# print ("reflector depth: ", ref_reflector_depth_1)
# print ("top layer velocity: ", ref_toplayer_vel_1)
# print ("primary reflector time = 1.5+ (2707/1886)*2 = 4.37s")

# plot_data_heatmap(refDataPlot2, ID = ref_ID_2, water_pert=ref_water_pert)
# print ("reflector depth: ", ref_reflector_depth_2)
# print ("top layer velocity: ", ref_toplayer_vel_2)
# print ("primary reflector time = 1.5+(2154/2354)*2 = 3.32s")

# plot_data_heatmap(refDataPlot3, ID = ref_ID_3, water_pert=ref_water_pert)
# print ("reflector depth: ", ref_reflector_depth_3)
# print ("top layer velocity: ", ref_toplayer_vel_3)
# print ("primary reflector time = 1.5+(1513/1998)*2 = 3.015s")

NameError: name 'Xo_val_ref_w' is not defined

In [6]:
#%% Load packages and setup GPU access
# resources: https://jhui.github.io/2017/03/07/TensorFlow-GPU/
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
tfk = tf.keras
tfkl = tf.keras.layers
tfkltd= tf.keras.layers.TimeDistributed

gpus = tf.config.list_physical_devices('GPU')
print("Tensorflow Version: ", tf.__version__)
print("Num GPUs Available: ", len(gpus))

2022-10-02 16:21:56.770639: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Tensorflow Version:  2.9.1
Num GPUs Available:  8


In [7]:
class NuisanceEncoder(tf.keras.Model):
  def __init__(self, kernel_sizes, filters, rdown=[2,2,1,1], tdown=[1,1,2,2], fstep=[2,4,8], latent_dim=7): #ori latent dimensions = 512
    super(NuisanceEncoder, self).__init__()
    k1,k2=kernel_sizes

    # self.c1=tfkltd(tfkl.Conv2D(filters,(k1,k2),activation='elu'))
    self.c1=tfkltd(tfkl.Conv2D(filters,(k1,k2),strides=(rdown[0],tdown[0]),activation='elu'))
    # self.mp1=tfkltd(tfkl.MaxPool2D(pool_size=(rdown[0],tdown[0])))
    # self.c3=tfkltd(tfkl.Conv2D(filters//fstep[0],(k1,k2),activation='elu'))
    self.c4=tfkltd(tfkl.Conv2D(filters//fstep[0],(k1,k2),strides=(rdown[1],tdown[1]),activation='elu'))
    # self.mp2=tfkltd(tfkl.MaxPool2D(pool_size=(rdown[1],tdown[1])))
    self.c5=tfkltd(tfkl.Conv2D(filters//fstep[1],(k1,k2),strides=(rdown[2],tdown[2]),activation='elu'))
    # self.c6=tfkltd(tfkl.Conv2D(filters//fstep[1],(k1,k2),activation='elu'))
    # self.mp3=tfkltd(tfkl.MaxPool2D(pool_size=(rdown[2],tdown[2])))
    # self.c7=tfkltd(tfkl.Conv2D(filters//fstep[2],(k1,k2),activation='elu'))
    self.c8=tfkltd(tfkl.Conv2D(filters//fstep[2],(k1,k2),strides=(rdown[3],tdown[3]),activation='elu'))
    self.bn=tfkltd(tfkl.BatchNormalization())
    # self.mp4=tfkltd(tfkl.MaxPool2D(pool_size=(rdown[3],tdown[3])))
    self.f=tfkltd(tfkl.Flatten())
    self.d=tfkltd(tfkl.Dense(latent_dim))

  def call(self, input_tensor, training=False):
    # n, ntau, nr, nt, nc = input_tensor.get_shape()
    x=self.c1(input_tensor)
    # x=self.c2(x)
    # x=self.mp1(x)
    # x=self.c3(x)
    x=self.c4(x)
    # x=self.mp2(x)
    x=self.c5(x)
    # x=self.c6(x)
    # x=self.mp3(x)
    # x=self.c7(x)
    x=self.c8(x)
    x=self.bn(x, training=training)
    # x=self.mp4(x)
    x=self.f(x)
    out=self.d(x)
    return out
  def model(self, x):
    return tfk.Model(inputs=x, outputs=self.call(x))


class SymmetricEncoder(tf.keras.Model):
  def __init__(self, kernel_sizes, filters, rdown=[2,2,2,1], tdown=[2,4,4,4],latent_dim=8, fstep=[2,4,8]):
    super(SymmetricEncoder, self).__init__()
    k1,k2=kernel_sizes

    self.c11=tfkltd(tfkl.Conv2D(filters,(k1,k2),activation='elu'))
    self.c12=tfkltd(tfkl.Conv2D(filters//fstep[0],(k1,k2),strides=(rdown[0],tdown[0]),activation='elu'))
    # self.mp11=tfkltd(tfkl.MaxPool2D(pool_size=(rdown[0],tdown[0])))
    self.c13=tfkltd(tfkl.Conv2D(filters//fstep[1],(k1,k2),activation='elu'))
    self.c14=tfkltd(tfkl.Conv2D(filters//fstep[2],(k1,k2),strides=(rdown[1],tdown[1]),activation='elu'))
    # self.mp12=tfkltd(tfkl.MaxPool2D(pool_size=(rdown[1],tdown[1])))


    self.c21=tfkl.Conv2D(filters,(k1,k2),strides=(rdown[2],tdown[2]),activation='elu')
    # self.c22=tfkl.Conv2D(filters//fstep[0],(k1,k2),padding='same',activation='elu')
    # self.mp21=tfkl.MaxPool2D(pool_size=(rdown[2],tdown[2]))
    # self.c23=tfkl.Conv2D(filters//fstep[1],(k1,k2),padding='same',activation='elu')
    self.c24=tfkl.Conv2D(filters//fstep[2],(k1,k2),strides=(rdown[3],tdown[3]),activation='elu')
    self.bn=tfkl.BatchNormalization()
    # self.mp22=tfkl.MaxPool2D(pool_size=(rdown[3],tdown[3]))
    self.f=tfkl.Flatten()
    self.d=tfkl.Dense(latent_dim)

  def call(self, input_tensor, training=False):
    # n, ntau, nr, nt, nc = input_tensor.get_shape()
    x=self.c11(input_tensor)
    x=self.c12(x)
    # x=self.mp11(x)
    x=self.c13(x)
    x=self.c14(x)
    # x=self.mp12(x)
    x=tf.math.reduce_mean(x,axis=1)
    x=self.c21(x)
    # x=self.c22(x)
    # x=self.mp21(x)
    # x=self.c23(x)
    x=self.c24(x)
    x=self.bn(x, training=training)
    # x=self.mp22(x)
    x=self.f(x)
    out=self.d(x)
    return out
  def model(self, x):
    return tfk.Model(inputs=x, outputs=self.call(x))


class DistributeZsym(tf.keras.Model):
  def __init__(self, ntau, nz0, nzi):
    super(DistributeZsym, self).__init__()

    self.nz0=nz0
    self.nzi=nzi
    self.ntau=ntau
    self.ri=tfkl.Reshape(target_shape=(ntau,nzi))
    self.repeat=tfkl.RepeatVector(ntau)
#     Xhatr=tfkl.Reshape(target_shape=(ntau,latent_dimr))(Xhatr)

  def call(self, z, training=False):

    z0,zi=tf.split(z,[self.nz0, self.ntau*self.nzi],axis=1)
    zi=self.ri(zi)
    z0=self.repeat(z0)
    out=tfkl.concatenate([z0, zi],axis=2)
    return out
  def model(self, x):
    return tfk.Model(inputs=x, outputs=self.call(x))


class LatentCat(tf.keras.Model): #concatenates zsy, and znuisance with dropout. #dropout performed to create more noise -> a form of stochastic regularizatino to make training more noisy. 
  def __init__(self, alpha=1.0):
    super(LatentCat, self).__init__()

    self.drop=tfkl.GaussianDropout(alpha)
    # self.drop=tfkl.Dropout(alpha)

  def call(self, zsym, znuisance,training=False):
    znuisance=self.drop(znuisance,training=training)
    z=tfkl.concatenate([zsym, znuisance])
    return z


class Mixer(tf.keras.Model):
  def __init__(self, kernel_sizes, filters, upfacts, nt, nr, fstep=[8,4,2]):
    super(Mixer, self).__init__()
    k1,k2=kernel_sizes
    rup,tup=upfacts

    filt_in=64

    self.d1=tfkltd(tfkl.Dense(units=((nr//rup)*(nt//tup)*filt_in),activation='elu'))
    self.r2=tfkltd(tfkl.Reshape(target_shape=((nr//rup),(nt//tup),filt_in)))
    self.c1=tfkltd(tfkl.Conv2D(filters//fstep[0],(k1,k2),padding='same',activation='elu'))
    self.us1=tfkltd(tfkl.UpSampling2D(size=(rup,tup)))
    # self.c2=tfkltd(tfkl.Conv2D(filters//fstep[1],(k1,k2),padding='same',activation='elu'))
    self.c3=tfkltd(tfkl.Conv2D(filters//fstep[1],(k1,k2),padding='same',activation='elu'))
    self.bn=tfkltd(tfkl.BatchNormalization())
    self.c4=tfkltd(tfkl.Conv2D(filters//fstep[2],(k1,k2),padding='same',activation='elu'))
    # self.c5=tfkltd(tfkl.Conv2D(filters//fstep[2],(k1,k2),padding='same',activation='elu'))
    self.c6=tfkltd(tfkl.Conv2D(1,(k1,k2),padding='same'))

  def call(self, z, training=False):
    x=self.d1(z)
    x=self.r2(x)
    x=self.us1(x)
    x=self.c1(x)
    # x=self.c2(x)
    x=self.c3(x)
    x=self.bn(x, training=training)
    x=self.c4(x)
    # x=self.c5(x)
    out=self.c6(x)
    return out
  def model(self, x):
    return tfk.Model(inputs=x, outputs=self.call(x))

In [8]:
encoder_input=tfk.Input(shape=(ntau,n1,n2,1), dtype='float32', name='encoder_input') #input shape => 10, 50, 410 = ntau,n1,n2 ###NONE
kernel_size=(5,5)
nfilt=64 #64
nz0=8 #symmetric encoder latent dimensions
nzi=22 #nuisance encoder latent dimensions. 


symencoder1=SymmetricEncoder(kernel_size,nfilt,[2,2,2,2],[2,2,2,2],nz0, [1,1,1]) #initialize model
xz0=symencoder1(encoder_input)
z0=tfkl.Flatten()(xz0)
nz0=z0.get_shape().as_list()[1]
symencoder=tfk.Model(encoder_input, z0, name='symencoder')
symencoder.summary()

nencoder1=NuisanceEncoder(kernel_size,nfilt,[2,2,2,2],[2,2,2,2],[1,1,1],nzi) #initialize model
xzi=nencoder1(encoder_input)
zi=tfkl.Flatten()(xzi)
nencoder=tfk.Model(encoder_input, zi, name='nencoder')
nencoder.summary()

decoder_input = tfk.Input(shape=(nzi*ntau+nz0), name='latentcode') #shape=(nzi*ntau+nz0) ###NONE

distzsym1=DistributeZsym(ntau, nz0, nzi) #initialize model
xdhat=distzsym1(decoder_input)
distzsym = tfk.Model(decoder_input, xdhat, name='distzsym')
distzsym.summary()


mixer_input = tfk.Input(shape=(ntau,nzi+nz0), name='mixer_input') #DOES FUSING OF COHERENT INFORMATION WITH EACH INSTANCE OF THE NUISANCE INFORMATION.just a placeholder for mixer1. shape=(ntau,nzi+nz0) ###NONE

mixer1=Mixer(kernel_size,nfilt,(10,10),n2,n1,[1,1,1]) #initialize model
xhat=mixer1(xdhat) #mixer_input #xdhat
mixer = tfk.Model(xdhat, xhat, name='mixer') #(mixer_input, xhat) #xdhat
mixer.summary()

znuisance=nencoder(encoder_input)
zsym=symencoder(encoder_input)
#xdist = distzsym(decoder_input)
#xpredict = mixer(xdhat)

#     report(upsampler,"upsampler")



NameError: name 'ntau' is not defined

In [ ]:
latentcat=LatentCat(0.65) #0.4 droupout

encoder=tfk.Model(encoder_input, latentcat(zsym,znuisance), name="encoder") #latentcat(zsym,znuisance)
encoder.summary()
#tfk.utils.plot_model(encoder, show_shapes=True)

decoder=tfk.Model(decoder_input, xhat, name="decoder") #xpredict
decoder.summary()

In [ ]:
# Clone model structure and Load weights
#-------------------------------------------------------------------------------------------------------------------
#Set flags
save_weights = False
load_weights = True
w_fn = "chosen_temp_weights_120E_22nenc_0.65D_6.6vel"
#-------------------------------------------------------------------------------------------------------------------
#Save weights
#print("Previous Dropout "+str(model.layers[2].layers[-1].rate))

if save_weights: 
  fn = "/content/drive/MyDrive/seismicTL_circular1_Brindha/Weights/" + w_fn
  model.save_weights(fn)

#new_dropout=0.5
#model.layers[2].layers[-1].rate=new_dropout
#model = tfk.models.clone_model(model)
#model.compile(optimizer='adam', loss='mse')

if load_weights:
  model=tf.keras.Model(encoder_input, decoder(latentcat(zsym,znuisance)) , name='autoencoder_clone')
  model.summary()
  model.compile(optimizer='adam', loss='mse')
  fn = "/content/drive/MyDrive/seismicTL_circular1_Brindha/Weights/" + w_fn
  model.load_weights(fn)

#print("New Dropout "+str(model.layers[2].layers[-2].rate))

In [ ]:
#Extract reference codes
ten_identical_ref_gathers = np.tile(reference_gather_1, (10,1,1,1)) 
nuisance_reference = nencoder.predict(ten_identical_ref_gathers)
#Extract wide validation codes
coherent_wide = symencoder.predict(Xo_val_w[:,:,:,:,:])
#Extract narrow validation codes
coherent_narrow = symencoder.predict(Xo_val_n[:,:,:,:,:])
#Redatum Codes
nuisance_reference_tiled = np.tile(nuisance_reference, (100,1))
hybrid_latent_wide = np.concatenate((coherent_wide,nuisance_reference_tiled),axis=1)
hybrid_latent_narrow = np.concatenate((coherent_narrow,nuisance_reference_tiled),axis=1)
corrected_wide=decoder.predict(hybrid_latent_wide)
corrected_narrow=decoder.predict(hybrid_latent_narrow)

In [ ]:
#Apply windowed cross-corelation on perturbed vs reference on model 901. 
#https://towardsdatascience.com/computing-cross-correlation-between-geophysical-time-series-488642be7bf0
import pandas as pd
import seaborn as sns
import scipy.stats as stats
#https://jinhyuncheong.com/jekyll/update/2019/05/16/Four_ways_to_qunatify_synchrony.html
def crosscorr(datax, datay, lag=0, wrap=False):
    """ Lag-N cross correlation.
    Shifted data filled with NaNs

    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length
    Returns
    ----------
    crosscorr : float
    """
    if wrap:
        shiftedy = datay.shift(lag)
        shiftedy.iloc[:lag] = datay.iloc[-lag:].values
        return datax.corr(shiftedy)
    else:
        return datax.corr(datay.shift(lag))

#Set Paramaters
##-----------------------------------------------------------------------------------------------------------------------------------------
dt = (8.84565-1.121)/559
print("dt:", dt)
time = np.linspace(start=1.121, stop=8.84565, num=560)
mn =5
inst =9 #+6% instance
tr = 49
window = 20
step_size = 1

#Run Code
##-----------------------------------------------------------------------------------------------------------------------------------------
refsg = Xo_val_ref_w[mn, 0, tr, :, 0]
p6sg = Xo_val_w[mn, inst, tr, :, 0]
df = pd.DataFrame
sr = pd.Series
window=window+2
t_start = 0
t_end = t_start + window
window_time = (window-2)*dt
rss=[]

while t_end < 560+window:
    d1 = sr(refsg).iloc[t_start:t_end]
    d2 = sr(p6sg).iloc[t_start:t_end]
    rs = np.nan_to_num([crosscorr(d1,d2, lag, wrap=False) for lag in range(-int(window/2)+1,int(window/2))])
    rss.append(rs)
    t_start = t_start + step_size
    t_end = t_end + step_size
rss = df(rss)


f,ax = plt.subplots(figsize=(12,8))
ax2 = sns.heatmap(rss,cmap='RdBu_r',ax=ax)
ax.set(title='Moving Window Cross-Correlation \n Reference vs Instance (+6.0%)',xlim=[0,window], xlabel='Lag (s)',ylabel='Time (s)')
ax.tick_params(axis='both', which='major', pad=5)
a = [np.fix(item-(window-1)/2) for item in ax.get_xticks()]
lags = [round(x * dt, 2) for x in a]
#ax.set_ylabel(labelpad=15)
ax.set_xticks(np.linspace(0,window-1,11))
#ax.set_xticklabels([np.fix(item-(window-1)/2) for item in ax.get_xticks()])
ax.set_xticklabels(lags)
ax.set_yticks(np.linspace(0,len(rss),10))
ax.set_yticklabels(np.around(np.linspace(start=1.121, stop=8.84565, num=10),3))
ax.margins=30
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(14)
ax.title.set_fontsize(17)
ax2.figure.axes[-1].set_ylabel('Normalized correlation coefficient', size=14)
plt.savefig("Correlation_Model901_Instance9_trace50_ref_perturbed6_title")

#seabed multiples
window_start = 23
window_start2 = 250 
window_start3 = 479
#reflector multiples
window_start4 = 282
window_start5 = 503
mp1 = int(window_start + (window-2)/2)
mp2 = int(window_start2 + (window-2)/2)
mp3 = int(window_start3 + (window-2)/2)
mp4 = int(window_start4 + (window-2)/2)
mp5 = int(window_start5 + (window-2)/2)
rss = rss.to_numpy()
shift1=np.argmax(rss[mp1,:])-(window-2)/2
shift2=np.argmax(rss[mp2,:])-(window-2)/2
shift3=np.argmax(rss[mp3,:])-(window-2)/2
shift4=np.argmax(rss[mp4,:])-(window-2)/2
shift5=np.argmax(rss[mp5,:])-(window-2)/2
ts1=np.around(shift1*dt, decimals=5)
ts2=np.around(shift2*dt, decimals=5)
ts3=np.around(shift3*dt, decimals=5)
ts4=np.around(shift4*dt, decimals=5)
ts5=np.around(shift5*dt, decimals=5)
print("time-shift1: ", np.around(shift1*dt, decimals=5))
print("time-shift2: ", np.around(shift2*dt, decimals=5))
print("time-shift3: ", np.around(shift3*dt, decimals=5))
print("time-shift4: ", np.around(shift4*dt, decimals=5))
print("time-shift5: ", np.around(shift5*dt, decimals=5))

In [ ]:
a = [np.fix(item-(window-1)/2) for item in ax.get_xticks()]
lags = [round(x * dt, 2) for x in a]
print(lags)

In [ ]:
#Set Paramaters
##-----------------------------------------------------------------------------------------------------------------------------------------
dt = (8.84565-1.121)/559
print("dt: ", dt)
time = np.linspace(start=1.121, stop=8.84565, num=560)
mn =5
inst =9 #+6% instance
tr = 49
window = 18 #40
step_size = 1

#Run Code
##-----------------------------------------------------------------------------------------------------------------------------------------
refsg = Xo_val_ref_w[mn, 0, tr, :, 0]
c6sg = corrected_wide[mn, inst, tr, :, 0]
df = pd.DataFrame
sr = pd.Series
window=window+2
t_start = 0
t_end = t_start + window
window_time = (window-2)*dt
rss=[]
while t_end < 560+window:
    d1 = sr(refsg).iloc[t_start:t_end]
    d2 = sr(c6sg).iloc[t_start:t_end]
    rs = np.nan_to_num([crosscorr(d1,d2, lag, wrap=False) for lag in range(-int(window/2)+1,int(window/2))])
    rss.append(rs)
    t_start = t_start + step_size
    t_end = t_end + step_size
rss = df(rss)

f,ax = plt.subplots(figsize=(12,8))
ax2 = sns.heatmap(rss,cmap='RdBu_r',ax=ax)
ax.set(title='Moving Window Cross-Correlation \n Reference vs Redatumed (+6.0%)',xlim=[0,window], xlabel='Offset (dt)',ylabel='Time (s)')
ax.tick_params(axis='both', which='major', pad=5)
ax.set_xticks(np.linspace(0,window-1,11))
#ax.set_xticklabels([np.fix(item-(window-1)/2) for item in ax.get_xticks()])
ax.set_xticklabels(lags)
ax.set_yticks(np.linspace(0,len(rss),10))
ax.set_yticklabels(np.around(np.linspace(start=1.121, stop=8.84565, num=10),3))
ax.margins=30
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(14)
ax.title.set_fontsize(17)
ax2.figure.axes[-1].set_ylabel('Normalized correlation coefficient', size=14)
plt.savefig("Correlation_Model901_Instance9_trace50_ref_redatumed6_title")

#seabed multiples
window_start = 23
window_start2 = 250 #228 #250
window_start3 = 479 #458 #479
#reflector multiples
window_start4 = 282
window_start5 = 503
mp1 = int(window_start + (window-2)/2)
mp2 = int(window_start2 + (window-2)/2)
mp3 = int(window_start3 + (window-2)/2)
mp4 = int(window_start4 + (window-2)/2)
mp5 = int(window_start5 + (window-2)/2)
rss = rss.to_numpy()
shift1=np.argmax(rss[mp1,:])-(window-2)/2
shift2=np.argmax(rss[mp2,:])-(window-2)/2
shift3=np.argmax(rss[mp3,:])-(window-2)/2
shift4=np.argmax(rss[mp4,:])-(window-2)/2
shift5=np.argmax(rss[mp5,:])-(window-2)/2
ts1=np.around(shift1*dt, decimals=5)
ts2=np.around(shift2*dt, decimals=5)
ts3=np.around(shift3*dt, decimals=5)
ts4=np.around(shift4*dt, decimals=5)
ts5=np.around(shift5*dt, decimals=5)
print("time-shift1: ", np.around(shift1*dt, decimals=5))
print("time-shift2: ", np.around(shift2*dt, decimals=5))
print("time-shift3: ", np.around(shift3*dt, decimals=5))
print("time-shift4: ", np.around(shift4*dt, decimals=5))
print("time-shift5: ", np.around(shift5*dt, decimals=5))


In [ ]:
np.argmax(rss[mp2,:])-(window-2)/2

In [ ]:
# import pandas as pd
# import seaborn as sns
# import scipy.stats as stats
# refsg = Xo_val_ref_w[mn, 0, tr, :, 0]
# p6sg = Xo_val_w[mn, inst, tr, :, 0]
# df = pd.DataFrame
# sr = pd.Series
# no_splits = 56
# samples_per_split = refsg.shape[0]/no_splits
# window=20
# rss=[]
# for t in range(0, no_splits):
#     d1 = sr(refsg).loc[(t)*samples_per_split:(t+1)*samples_per_split]
#     d2 = sr(p6sg).loc[(t)*samples_per_split:(t+1)*samples_per_split]
#     rs = np.nan_to_num([crosscorr(d1,d2, lag) for lag in range(-int(window/2),int(window/2))])
#     rss.append(rs) #list of list for each 

# f,ax = plt.subplots(figsize=(10,20))
# sns.heatmap(rss,cmap='RdBu_r',ax=ax)
# ax.set(title=f'(reference leads <) Windowed Time Lagged Cross Correlation (> perturbed leads)',xlim=[0,window], xlabel='Offset',ylabel='Window epochs')
# ax.set_xticklabels([int(item-window/2) for item in ax.get_xticks()]);


In [ ]:
#np.shape(corrected_narrow)
#Evaluate norms: 
from numpy import linalg as LA
def norm(data1, data2):
  '''data1 is true, data2 is corrected'''
  data1 = np.transpose(data1)
  data2 = np.transpose(data2)
  num = data1-data2
  denom = data1

  num = LA.norm(num, 'fro')
  denom = LA.norm(denom, 'fro')
  norm_ratio = num/denom
  return norm_ratio

! pip install wiggle
import wiggle as wiggle
def plot_wiggle(x,c='k', txt='none', t=None, label_flag='off'):
    import numpy as np
    import matplotlib.pyplot as plt
    # fig=plt.figure(dpi=150, figsize=(6,12))
    sf=np.max(np.std(x,axis=1))
    wiggle.wiggle(x.T, tt = t, color=c, sf= sf/0.5/10)#
    # if(txt != 'none'):
        # plt.text(0.5, 8, txt, size=20, color='black', bbox=dict(facecolor='red', alpha=1,))
    plt.axis(label_flag)
    # plt.savefig(os.path.join(dir,s), bbox_inches='tight')
    # plt.close()



In [ ]:
#Plot traces with the same subsurface for fine combing comparison
##-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#print("reference_gather_1.shape", reference_gather_1[0, 0, 0:1, :, 0])
print("corrected_narrow.shape", corrected_narrow.shape)
print("uncorrected narrow.shape", Xo_val_n[:,:,:,:,:].shape)
print("true/reference narrow.shape", Xo_val_ref_n[:,:,:,:,:].shape)
time = np.linspace(start=1.121, stop=8.84565, num=560)
colors = plt.cm.viridis_r(np.linspace(0,1,6)) #reference on colormaps: https://stackoverflow.com/questions/38208700/matplotlib-plot-lines-with-colors-through-colormap
col = 'purple'
plot_window= False
#time.shape = 560
# window_start2 = 228 #232  #248
# window_start3 = 458   #475

#Collect traces
mn= 5#5 #model_number

#trace0r.shape = (560,)
trace0r = Xo_val_ref_w[mn, 0, 0, :, 0]
trace49r = Xo_val_ref_w[mn, 0, 49, :, 0]
trace99r = Xo_val_ref_w[mn, 0, 99, :, 0]
#traceX.shape = (10,560)
trace0 = corrected_wide[mn, :, 0, :, 0]
trace49 = corrected_wide[mn, :, 49, :, 0]
trace99 = corrected_wide[mn, :, 99, :, 0]
#traceXp.shape = (10,560)
trace0p = Xo_val_w[mn, :, 0, :, 0]
trace49p = Xo_val_w[mn, :, 49, :, 0]
trace99p = Xo_val_w[mn, :, 99, :, 0]

fig1 = plt.figure(figsize=(30,7))
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.1, 
                    hspace=0.4)
plt.subplot(1,2,1)
plt.plot(trace0r, color='red', label=("Reference"))
# plt.plot(trace0p[0,:], color=colors[0], alpha=0.6,label=("-6.0% Instance"))
# plt.plot(trace0p[2,:], color=colors[1],alpha=0.6, label=("-5.0% Instance"))
# plt.plot(trace0p[4,:],color=colors[2], alpha=0.6,label=("-4.0% Instance"))
# plt.plot(trace0p[5,:],color=colors[3], alpha=0.6,label=("+4.0% Instance"))
# plt.plot(trace0p[7,:], color=colors[4],alpha=0.6, label=("+5.0% Instance"))
plt.plot(trace0p[9,:],color=colors[5], alpha=0.6,label=("+6.0% Instance"))
plt.xticks(np.linspace(0, 560, 10), np.around(np.linspace(start=1.121, stop=8.84565, num=10), 3))
plt.xlabel("Time (s)", fontsize=14, labelpad=15)
plt.ylabel("Standardized Waveform Amplitude", fontsize=14, labelpad=15)
plt.title('Perturbed Far Offset (-3.25km)', fontsize=17, pad=8)
plt.legend(fontsize=12)

plt.subplot(1,2,2)
plt.plot(trace0r, color='red', label=("Reference"))
# plt.plot(trace0[0,:], color=colors[0], alpha=0.6,label=("-6.0% Instance"))
# plt.plot(trace0[2,:], color=colors[1],alpha=0.6, label=("-5.0% Instance"))
# plt.plot(trace0[4,:],color=colors[2], alpha=0.6,label=("-4.0% Instance"))
# plt.plot(trace0[5,:],color=colors[3], alpha=0.6,label=("+4.0% Instance"))
# plt.plot(trace0[6,:], color=colors[4],alpha=0.6, label=("+5.0% Instance"))
plt.plot(trace0[7,:],color=colors[5], alpha=0.6,label=("+6.0% Instance"))
plt.xticks(np.linspace(0, 560, 10), np.around(np.linspace(start=1.121, stop=8.84565, num=10), 3))
plt.xlabel("Time (s)", fontsize=14, labelpad=15)
plt.ylabel("Standardized Waveform Amplitude", fontsize=14, labelpad=15)
plt.title('Redatumed Far Offset (-3.25km)', fontsize=17, pad=8)
plt.legend(fontsize=12)
fname = 'Trace 1 Model 901'
#plt.savefig(fname)

fig2 = plt.figure(figsize=(30,7))
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.1, 
                    hspace=0.4)
plt.subplot(1,2,1)
plt.plot(trace49r, color='red', label=("Reference"))
# plt.plot(trace49p[0,:], color=colors[0], alpha=0.6,label=("-6.0% Instance"))
# plt.plot(trace49p[2,:], color=colors[1],alpha=0.6, label=("-5.0% Instance"))
# plt.plot(trace49p[4,:],color=colors[2], alpha=0.6,label=("-4.0% Instance"))
# plt.plot(trace49p[5,:],color=colors[3], alpha=0.6,label=("+4.0% Instance"))
# plt.plot(trace49p[7,:], color=colors[4],alpha=0.6, label=("+5.0% Instance"))
plt.plot(trace49p[9,:],color=colors[5], alpha=0.6,label=("+6.0% Instance"))

plt.text(window_start+(window+10), 6, r'$\Delta t_{1} = $'+str(ts1) + 's', fontsize=15)
plt.text(window_start2+(window-45), 6, r'$\Delta t_{3} = $'+str(ts2) + 's', fontsize=15)      #window-7 window-40
plt.text(window_start3+(window-50), 6, r'$\Delta t_{5} = $'+str(ts3) + 's', fontsize=15)      #window-24 window-40
plt.text(window_start4+(window-30), -4.5, r'$\Delta t_{2} = $'+str(ts4) + 's', fontsize=15)      #window-24 window-40
plt.text(window_start5+(window-28), -4.5, r'$\Delta t_{4} = $'+str(ts5) + 's', fontsize=15)      #window-24 window-40

if plot_window:
  plt.plot([window_start,window_start+(window-2)],[2.5,2.5],'blue',linestyle = 'dotted', lw=5)
  plt.vlines(window_start, -0.2, 2.5, colors='blue', linestyles='solid', lw=5)
  plt.vlines(window_start+(window-2), -0.2, 2.5, colors='blue', linestyles='solid', lw=5) 
  # plt.plot([window_start2,window_start2+(window-2)],[2.5,2.5],'blue',linestyle = 'dotted', lw=5, label=("Cross-Correlation Window"))
  # plt.vlines(window_start2, -0.2, 2.5, colors='blue', linestyles='solid', lw=5)
  # plt.vlines(window_start2+(window-2), -0.2, 2.5, colors='blue', linestyles='solid', lw=5)
  # plt.plot([window_start3,window_start3+(window-2)],[2.5,2.5],'blue',linestyle = 'dotted', lw=5)
  # plt.vlines(window_start3, -0.2, 2.5, colors='blue', linestyles='solid', lw=5)
  # plt.vlines(window_start3+(window-2), -0.2, 2.5, colors='blue', linestyles='solid', lw=5)
  # plt.plot([window_start4,window_start4+(window-2)],[-2.5,-2.5],'blue',linestyle = 'dotted', lw=5)
  # plt.vlines(window_start4, 0.0, -2.5, colors='blue', linestyles='solid', lw=5)
  # plt.vlines(window_start4+(window-2), 0.0, -2.5, colors='blue', linestyles='solid', lw=5)
  # plt.plot([window_start5,window_start5+(window-2)],[-2.5,-2.5],'blue',linestyle = 'dotted', lw=5)
  # plt.vlines(window_start5, 0.0, -2.5, colors='blue', linestyles='solid', lw=5)
  # plt.vlines(window_start5+(window-2), 0.0, -2.5, colors='blue', linestyles='solid', lw=5)

plt.xlabel("Time (s)", fontsize=14, labelpad=15)
plt.ylabel("Standardized Waveform Amplitude", fontsize=14, labelpad=15)
plt.xticks(np.linspace(0, 560, 10), np.around(np.linspace(start=1.121, stop=8.84565, num=10), 3))
plt.title('Zero Offset', fontsize=17, pad=8)
plt.legend(fontsize=12)

plt.subplot(1,2,2)
plt.plot(trace49r, color='red', label=("Reference"))
# plt.plot(trace49[0,:], color=colors[0], alpha=0.6,label=("-6.0% Instance"))
# plt.plot(trace49[2,:], color=colors[1],alpha=0.6, label=("-5.0% Instance"))
# plt.plot(trace49[4,:],color=colors[2], alpha=0.6,label=("-4.0% Instance"))
# plt.plot(trace49[5,:],color=colors[3], alpha=0.6,label=("+4.0% Instance"))
# plt.plot(trace49[7,:], color=colors[4],alpha=0.6, label=("+5.0% Instance"))
plt.plot(trace49[9,:],color=colors[5], alpha=0.6,label=("+6.0% Redatumed"))

plt.text(window_start+(window+10), 6, r'$\Delta t_{1} = $'+str(0.00) + 's', fontsize=15)
plt.text(window_start2+(window-45), 6, r'$\Delta t_{3} = $'+str(0.00) + 's', fontsize=15)      #window-7 window-40
plt.text(window_start3+(window-50), 6, r'$\Delta t_{5} = $'+str(0.00) + 's', fontsize=15)      #window-24 window-40
plt.text(window_start4+(window-30), -4.5, r'$\Delta t_{2} = $'+str(0.00) + 's', fontsize=15)      #window-24 window-40
plt.text(window_start5+(window-28), -4.5, r'$\Delta t_{4} = $'+str(0.00) + 's', fontsize=15)      #window-24 window-40


if plot_window:
  plt.plot([window_start,window_start+(window-2)],[2.5,2.5],'blue',linestyle = 'dotted', lw=5)
  plt.vlines(window_start, -0.2, 2.5, colors='blue', linestyles='solid', lw=5)
  plt.vlines(window_start+(window-2), -0.2, 2.5, colors='blue', linestyles='solid', lw=5)
  plt.plot([window_start2,window_start2+(window-2)],[2.5,2.5],'blue',linestyle = 'dotted', lw=5, label=("Cross-Correlation Window"))
  plt.vlines(window_start2, -0.2, 2.5, colors='blue', linestyles='solid', lw=5)
  plt.vlines(window_start2+(window-2), -0.2, 2.5, colors='blue', linestyles='solid', lw=5)
  plt.plot([window_start3,window_start3+(window-2)],[2.5,2.5],'blue',linestyle = 'dotted', lw=5)
  plt.vlines(window_start3, -0.2, 2.5, colors='blue', linestyles='solid', lw=5)
  plt.vlines(window_start3+(window-2), -0.2, 2.5, colors='blue', linestyles='solid', lw=5)
  plt.plot([window_start4,window_start4+(window-2)],[-2.5,-2.5],'blue',linestyle = 'dotted', lw=5)
  plt.vlines(window_start4, 0.0, -2.5, colors='blue', linestyles='solid', lw=5)
  plt.vlines(window_start4+(window-2), 0.0, -2.5, colors='blue', linestyles='solid', lw=5)
  plt.plot([window_start5,window_start5+(window-2)],[-2.5,-2.5],'blue',linestyle = 'dotted', lw=5)
  plt.vlines(window_start5, 0.0, -2.5, colors='blue', linestyles='solid', lw=5)
  plt.vlines(window_start5+(window-2), 0.0, -2.5, colors='blue', linestyles='solid', lw=5)

plt.xlabel("Time (s)", fontsize=14, labelpad=15)
plt.ylabel("Standardized Waveform Amplitude", fontsize=14, labelpad=15)
plt.xticks(np.linspace(0, 560, 10), np.around(np.linspace(start=1.121, stop=8.84565, num=10), 3))
plt.title('Zero Offset', fontsize=17, pad=8)
plt.legend(fontsize=12)
fname = 'Trace 50 Model 906 Timeshifts'
plt.savefig(fname)

fig3 = plt.figure(figsize=(30,7))
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.1, 
                    hspace=0.4)
plt.subplot(1,2,1)
plt.plot(trace99r, color='red', label=("Reference"))
plt.text(window_start+(window-120), 6, r'$\Delta t_{1} = $'+str(ts1) + 's', fontsize=15)
plt.text(window_start2+(window-50), 6, r'$\Delta t_{3} = $'+str(ts2) + 's', fontsize=15)      #window-7 window-40
plt.text(window_start3+(window-46), 6, r'$\Delta t_{5} = $'+str(ts3) + 's', fontsize=15)      #window-24 window-40
plt.text(window_start4+(window-39), -4.5, r'$\Delta t_{2} = $'+str(ts4) + 's', fontsize=15)      #window-24 window-40
plt.text(window_start5+(window-50), -4.5, r'$\Delta t_{4} = $'+str(ts5) + 's', fontsize=15)     
# plt.plot([window_start,window_start+(window-2)],[2.5,2.5],'blue',linestyle = 'dotted', lw=5)
# plt.vlines(window_start, -0.2, 2.5, colors='blue', linestyles='solid', lw=5)
# plt.vlines(window_start+(window-2), -0.2, 2.5, colors='blue', linestyles='solid', lw=5)
# plt.plot(trace99p[0,:], color=colors[0], alpha=0.6,label=("-6.0% Instance"))
# plt.plot(trace99p[2,:], color=colors[1],alpha=0.6, label=("-5.0% Instance"))
# plt.plot(trace99p[4,:],color=colors[2], alpha=0.6,label=("-4.0% Instance"))
# plt.plot(trace99p[5,:],color=colors[3], alpha=0.6,label=("+4.0% Instance"))
# plt.plot(trace99p[7,:], color=colors[4],alpha=0.6, label=("+5.0% Instance"))
plt.plot(trace99p[9,:],color=colors[5], alpha=0.6,label=("+6.0% Instance"))
plt.xlabel("Time (s)", fontsize=14, labelpad=15)
plt.ylabel("Standardized Waveform Amplitude", fontsize=14, labelpad=15)
plt.xticks(np.linspace(0, 560, 10), np.around(np.linspace(start=1.121, stop=8.84565, num=10), 3))
plt.title('Far Offset (+3.25km)', fontsize=17, pad=8)
plt.legend(fontsize=12)

plt.subplot(1,2,2)
plt.plot(trace99r, color='red', label=("Reference"))
plt.text(window_start+(window-120), 6, r'$\Delta t_{1} = $'+str(0.00) + 's', fontsize=15)
plt.text(window_start2+(window-50), 6, r'$\Delta t_{3} = $'+str(0.00) + 's', fontsize=15)      #window-7 window-40
plt.text(window_start3+(window-46), 6, r'$\Delta t_{5} = $'+str(0.00) + 's', fontsize=15)      #window-24 window-40
plt.text(window_start4+(window-35), -4.5, r'$\Delta t_{2} = $'+str(0.01382) + 's', fontsize=15)      #window-24 window-40
plt.text(window_start5+(window-35), -4.5, r'$\Delta t_{4} = $'+str(0.00) + 's', fontsize=15) 
# plt.plot(trace99[0,:], color=colors[0], alpha=0.6,label=("-6.0% Instance"))
# plt.plot(trace99[2,:], color=colors[1],alpha=0.6, label=("-5.0% Instance"))
# plt.plot(trace99[4,:],color=colors[2], alpha=0.6,label=("-4.0% Instance"))
# plt.plot(trace99[5,:],color=colors[3], alpha=0.6,label=("+4.0% Instance"))
# plt.plot(trace99[7,:], color=colors[4],alpha=0.6, label=("+5.0% Instance"))
plt.plot(trace99[9,:],color=colors[5], alpha=0.6,label=("+6.0% Instance"))
plt.xlabel("Time (s)", fontsize=14, labelpad=15)
plt.ylabel("Standardized Waveform Amplitude", fontsize=14, labelpad=15)
plt.xticks(np.linspace(0, 560, 10), np.around(np.linspace(start=1.121, stop=8.84565, num=10), 3))
plt.title('Far Offset (+3.25km)', fontsize=17, pad=8)
plt.legend(fontsize=12)
fname = 'Trace 100 Model 906 timeshifts'
#plt.savefig(fname)

In [ ]:
#Plot traces across different subsurfaces to see if seabed multiples arrive at the same time, albeit with different amplitudes
##-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#print("reference_gather_1.shape", reference_gather_1[0, 0, 0:1, :, 0])
print("corrected_narrow.shape", corrected_narrow.shape)
print("uncorrected narrow.shape", Xo_val_n[:,:,:,:,:].shape)
print("true/reference narrow.shape", Xo_val_ref_n[:,:,:,:,:].shape)
time = np.linspace(start=1.121, stop=8.84565, num=560)
colors = plt.cm.viridis_r(np.linspace(0,1,6)) #reference on colormaps: https://stackoverflow.com/questions/38208700/matplotlib-plot-lines-with-colors-through-colormap
#time.shape = 560

#Collect traces
trace0p = Xo_val_w[0, 0, 0, :, 0]
# trace0 = corrected_wide[0:nmodels-1, 0, 0, :, 0]
# trace49 = corrected_wide[0:nmodels-1, 0, 49, :, 0]
# trace99 = corrected_wide[0:nmodels-1, 0, 99, :, 0]

nmodels = 6
Index = [0,3,6, 9, 3, 6]
for i in range(nmodels): 
  it0 = corrected_wide[i, Index[i], 0:1, :, 0]
  it0p = Xo_val_w[i, Index[i], 0:1, :, 0]
  it49 = corrected_wide[i, Index[i], 49:50, :, 0]
  it49p = Xo_val_w[i, Index[i], 49:50, :, 0]
  it99 = corrected_wide[i, Index[i], 99:100, :, 0]
  it99p = Xo_val_w[i, Index[i], 99:100, :, 0]
  if i == 0: 
    trace0 = it0
    trace0p = it0p
    trace49 = it49
    trace49p = it49p
    trace99 = it99
    trace99p = it99p
  else:
    trace0 = np.append(trace0, it0, axis=0)
    trace0p = np.append(trace0p, it0p, axis=0)
    trace49 = np.append(trace49, it49, axis=0)
    trace49p = np.append(trace49p, it49p, axis=0)
    trace99 = np.append(trace99, it99, axis=0)
    trace99p = np.append(trace99p, it99p, axis=0)


fig1 = plt.figure(figsize=(30,7))
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.1, 
                    hspace=0.4)
plt.subplot(1,2,1)
plt.plot(reference_gather_1[0, 0, 0, :, 0], color='black', label=("Auxiliary Record"))
plt.plot(trace0p[0,:], color=colors[0], alpha=0.8,label=("-6.0% Instance"))
plt.plot(trace0p[1,:], color=colors[1],alpha=0.8, label=("-4.5% Instance"))
plt.plot(trace0p[4,:],color=colors[2], alpha=0.8,label=("-3.0% Instance"))
plt.plot(trace0p[5,:],color=colors[3], alpha=0.8,label=("+3.0% Instance"))
plt.plot(trace0p[2,:], color=colors[4],alpha=0.8, label=("+4.5% Instance"))
plt.plot(trace0p[3,:],color=colors[5], alpha=0.8,label=("+6.0% Instance"))
plt.xticks(np.linspace(0, 560, 10), np.around(np.linspace(start=1.121, stop=8.84565, num=10), 3))
plt.xlabel("Time (s)", fontsize=14, labelpad=15)
plt.ylabel("Standardized Waveform Amplitude", fontsize=14, labelpad=15)
plt.title('Perturbed Far Offset (-3.25km)', fontsize=17, pad=8)
plt.legend(fontsize=12)

plt.subplot(1,2,2)
plt.plot(reference_gather_1[0, 0, 0, :, 0], color='black', label=("Auxiliary Record"))
plt.plot(trace0[0,:], color=colors[0], alpha=0.8,label=("-6.0% Instance"))
plt.plot(trace0[1,:], color=colors[1],alpha=0.8, label=("-4.5% Instance"))
plt.plot(trace0[4,:],color=colors[2], alpha=0.8,label=("-3.0% Instance"))
plt.plot(trace0[5,:],color=colors[3], alpha=0.8,label=("+3.0% Instance"))
plt.plot(trace0[2,:], color=colors[4],alpha=0.8, label=("+4.5% Instance"))
plt.plot(trace0[3,:],color=colors[5], alpha=0.8,label=("+6.0% Instance"))
plt.xticks(np.linspace(0, 560, 10), np.around(np.linspace(start=1.121, stop=8.84565, num=10), 3))
plt.xlabel("Time (s)", fontsize=14, labelpad=15)
plt.ylabel("Standardized Waveform Amplitude", fontsize=14, labelpad=15)
plt.title('Redatumed Far Offset (-3.25km)', fontsize=17, pad=8)
plt.legend(fontsize=12)
fname = 'Trace 1'
plt.savefig(fname)

fig2 = plt.figure(figsize=(30,7))
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.1, 
                    hspace=0.4)
plt.subplot(1,2,1)
plt.plot(reference_gather_1[0, 0, 49, :, 0], color='black', label=("Reference"))
plt.plot(trace49p[0,:], color=colors[0], alpha=0.8, label=("Test 1"))
plt.plot(trace49p[1,:], color=colors[1], alpha=0.8, label=("Test 2"))
plt.plot(trace49p[2,:], color=colors[2], alpha=0.8,label=("Test 3"))
plt.plot(trace49p[3,:],color=colors[3],alpha=0.8, label=("Test 4"))
plt.plot(trace49p[4,:],color=colors[4], alpha=0.8, label=("Test 5"))
plt.plot(trace49p[5,:],color=colors[5], alpha=0.8,label=("Test 6"))
plt.xlabel("Time (s)", fontsize=14, labelpad=15)
plt.ylabel("Standardized Waveform Amplitude", fontsize=14, labelpad=15)
plt.xticks(np.linspace(0, 560, 10), np.around(np.linspace(start=1.121, stop=8.84565, num=10), 3))
plt.title('Zero Offset', fontsize=17, pad=8)
plt.legend(fontsize=12)

plt.subplot(1,2,2)
plt.plot(reference_gather_1[0, 0, 49, :, 0], color='black', label=("Reference"))
plt.plot(trace49[0,:], color=colors[0], alpha=0.8,label=("Test 1"))
plt.plot(trace49[1,:], color=colors[1], alpha=0.8,label=("Test 2"))
plt.plot(trace49[2,:], color=colors[2],alpha=0.8, label=("Test 3"))
plt.plot(trace49[3,:],color=colors[3], alpha=0.8,label=("Test 4"))
plt.plot(trace49[4,:],color=colors[4], alpha=0.8,label=("Test 5"))
plt.plot(trace49[5,:],color=colors[5], alpha=0.8,label=("Test 6"))
plt.xlabel("Time (s)", fontsize=14, labelpad=15)
plt.ylabel("Standardized Waveform Amplitude", fontsize=14, labelpad=15)
plt.xticks(np.linspace(0, 560, 10), np.around(np.linspace(start=1.121, stop=8.84565, num=10), 3))
plt.title('Zero Offset', fontsize=17, pad=8)
plt.legend(fontsize=12)
fname = 'Trace 50'
#plt.savefig(fname)

fig3 = plt.figure(figsize=(30,7))
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.1, 
                    hspace=0.4)
plt.subplot(1,2,1)
plt.plot(reference_gather_1[0, 0, 0, :, 0], color='black', label=("Reference"))
plt.plot(trace99p[0,:], color=colors[0], alpha=0.8,label=("Test 1"))
plt.plot(trace99p[1,:], color=colors[1], alpha=0.8,label=("Test 2"))
plt.plot(trace99p[2,:], color=colors[2], alpha=0.8,label=("Test 3"))
plt.plot(trace99p[3,:],color=colors[3], alpha=0.8,label=("Test 4"))
plt.plot(trace99p[4,:],color=colors[4], alpha=0.8,label=("Test 5"))
plt.plot(trace99p[5,:],color=colors[5], alpha=0.8,label=("Test 6"))
plt.xlabel("Time (s)", fontsize=14, labelpad=15)
plt.ylabel("Standardized Waveform Amplitude", fontsize=14, labelpad=15)
plt.xticks(np.linspace(0, 560, 10), np.around(np.linspace(start=1.121, stop=8.84565, num=10), 3))
plt.title('Far Offset (+3.25km)', fontsize=17, pad=8)
plt.legend(fontsize=12)

plt.subplot(1,2,2)
plt.plot(reference_gather_1[0, 0, 0, :, 0], color='black', label=("Reference"))
plt.plot(trace99[0,:], color=colors[0], alpha=0.8,label=("Test 1"))
plt.plot(trace99[1,:], color=colors[1], alpha=0.8,label=("Test 2"))
plt.plot(trace99[2,:], color=colors[2], alpha=0.8,label=("Test 3"))
plt.plot(trace99[3,:],color=colors[3], alpha=0.8,label=("Test 4"))
plt.plot(trace99[4,:],color=colors[4], alpha=0.8,label=("Test 5"))
plt.plot(trace99[5,:],color=colors[5], alpha=0.8,label=("Test 6"))
plt.xlabel("Time (s)", fontsize=14, labelpad=15)
plt.ylabel("Standardized Waveform Amplitude", fontsize=14, labelpad=15)
plt.xticks(np.linspace(0, 560, 10), np.around(np.linspace(start=1.121, stop=8.84565, num=10), 3))
plt.title('Far Offset (+3.25km)', fontsize=17, pad=8)
plt.legend(fontsize=12)
fname = 'Trace 100'
#plt.savefig(fname)

In [ ]:
#------------------------------------------------------------------------------------------------------------------------
#Plot uncorrected val gather and reference velocity gather side by side
#Plot corrected/hybrid validation gather underneath and true reference validation gather. 

#Set flags - pick data point: Instance numbers
time = np.linspace(start=1.121, stop=8.84565, num=560)
plot_data_heatmap(Xo_val_ref_w[0,0,:,:,0] - corrected_wide[0,7,:,:,0])
plot_wiggle(corrected_wide[0,7,:,:,0], t=time, label_flag='on')

# New Section

In [ ]:
#View Corrections
#------------------------------------------------------------------------------------------------------------------------
#Decide on format
! pip install wiggle
import wiggle
#------------------------------------------------------------------------------------------------------------------------
#Define plot_wiggle function
def plot_wiggle(x,c='k', txt='none', t=None, label_flag='off'):
    import numpy as np
    import wiggle as wiggle
    import matplotlib.pyplot as plt
    # fig=plt.figure(dpi=150, figsize=(6,12))
    sf=np.max(np.std(x,axis=1))
    wiggle.wiggle(x.T, tt = t, color=c, sf= sf/0.5/10)#
    # if(txt != 'none'):
        # plt.text(0.5, 8, txt, size=20, color='black', bbox=dict(facecolor='red', alpha=1,))
    plt.axis(label_flag)
    # plt.savefig(os.path.join(dir,s), bbox_inches='tight')
    # plt.close()

#------------------------------------------------------------------------------------------------------------------------
#Plot uncorrected val gather and reference velocity gather side by side
#Plot corrected/hybrid validation gather underneath and true reference validation gather. 

#Set flags - pick data point: Instance numbers
instances = [0,1,2] #6,7,8
time = np.linspace(start=1.121, stop=8.84565, num=560)
time.shape #560
#------------------------------------------------------------------------------------------------------------------------
#Information about data
ref_data = (ten_identical_ref_gathers[0, 5, :, :, 0]) #ten_identical_ref_gathers
#tval_data = (reference_val_gather_1[0, 0, :, :, 0])
pval_data_1 = (perturbed_val_gather_1[0, instances[0], :, :, 0])
pval_data_2 = (perturbed_val_gather_1[0, instances[1], :, :, 0])
pval_data_3 = (perturbed_val_gather_1[0, instances[2], :, :, 0])
cval_data_1 = (corrected_val_gathers_1[0, instances[0], :, :, 0])
cval_data_2 = (corrected_val_gathers_1[0, instances[1], :, :, 0])
cval_data_3 = (corrected_val_gathers_1[0, instances[2], :, :, 0])
val_water_pert_1 = round(labels_val[val_sample][str(instances[0])]['water_pertubation'], 3)
val_water_pert_2 = round(labels_val[val_sample][str(instances[1])]['water_pertubation'], 3)
val_water_pert_3 = round(labels_val[val_sample][str(instances[2])]['water_pertubation'], 3)

#ID = labels_train[sample][str(instance)]['ID']
#water_pert = round(labels_train[sample][str(instance)]['water_pertubation'], 3)
#------------------------------------------------------------------------------------------------------------------------
#Plot data
fig1 = plt.figure(figsize=(15,7))

plt.subplot(1,2,1)
plt.title('Auxiliary record', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
plot_wiggle(ref_data, 'b')
plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)

plt.subplot(1,2,2)
plt.title('True Validation Model', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
plot_wiggle(tval_data)
plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)

fname = 'Reference and True Validation Model'
#plt.savefig(fname)
plt.show()

In [ ]:
fig2 = plt.figure(figsize=(30,21))

plt.subplot(3, 4, 1)
plt.title(str(val_water_pert_1) +  '% ' 'Perturbed Validation', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
plot_wiggle(pval_data_1, t=time, label_flag='on')
plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)

plt.subplot(3,4,2)
plt.title('Redatumed Validation', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
plot_wiggle(cval_data_1, 'b', t=time, label_flag='on')
plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)

plt.subplot(3, 4, 3)
plt.title('Reference Validation - Redatumed Validation', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
im1 = plt.imshow(np.transpose(cval_data_1-tval_data), aspect='auto', cmap='RdBu', extent = [0, 100, 8.846, 1.121])
#plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
#plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)
plt.axis("On")
cbar = fig1.colorbar(im1)
cbar.set_label('Waveform amplitude', fontsize = 15, labelpad=25, rotation=270)
plt.clim(-5,5)

plt.subplot(3, 4, 4)
plt.title('Reference Validation - Perturbed Validation', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
im1 = plt.imshow(np.transpose(pval_data_1-tval_data), aspect='auto', cmap='RdBu', extent = [0, 100, 8.846, 1.121])
#plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
#plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)
plt.axis("On")
cbar = fig1.colorbar(im1)
cbar.set_label('Waveform amplitude', fontsize = 15, labelpad=25, rotation=270)
plt.clim(-5,5)

plt.subplot(3, 4, 5)
plt.title(str(val_water_pert_2) +  '% ' 'Perturbed Validation', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
plot_wiggle(pval_data_2, t=time, label_flag='on')
plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)

plt.subplot(3, 4, 6)
plt.title('Redatumed Validation', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
plot_wiggle(cval_data_2, 'b', t=time, label_flag='on')
plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)

plt.subplot(3, 4, 7)
plt.title('Reference Validation - Redatumed Validation', fontsize = 15, pad =12)
#plt.tick_params(axis='both', labelsize=12)
im1 = plt.imshow(np.transpose(cval_data_2-tval_data), aspect='auto', cmap='RdBu', extent = [0, 100, 8.846, 1.121])
plt.axis("On")
#plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
#plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)
cbar = fig1.colorbar(im1)
cbar.set_label('Waveform amplitude', fontsize = 15, labelpad=25, rotation=270)
plt.clim(-5,5)

plt.subplot(3, 4, 8)
plt.title('Reference Validation - Perturbed Validation', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
im1 = plt.imshow(np.transpose(pval_data_2-tval_data), aspect='auto', cmap='RdBu', extent = [0, 100, 8.846, 1.121])
#plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
#plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)
plt.axis("On")
cbar = fig1.colorbar(im1)
cbar.set_label('Waveform amplitude', fontsize = 15, labelpad=25, rotation=270)
plt.clim(-5,5)

plt.subplot(3, 4, 9)
plt.title(str(val_water_pert_3) +  '% ' 'Perturbed Validation', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
plot_wiggle(pval_data_3, t=time, label_flag='on')
plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)

plt.subplot(3, 4, 10)
plt.title('Redatumed Validation', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
plot_wiggle(cval_data_3, 'b', t=time, label_flag='on')
plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)

plt.subplot(3, 4, 11)
plt.title('Reference Validation - Redatumed Validation', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
im1 = plt.imshow(np.transpose(cval_data_3-tval_data), aspect='auto', cmap='RdBu', extent = [0, 100, 8.846, 1.121])
#plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)
plt.axis("On")
cbar = fig1.colorbar(im1)
cbar.set_label('Waveform amplitude', fontsize = 15, labelpad=25, rotation=270)
plt.clim(-5,5)

plt.subplot(3, 4, 12)
plt.title('Reference Validation - Perturbed Validation', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
im1 = plt.imshow(np.transpose(pval_data_3-tval_data), aspect='auto', cmap='RdBu', extent = [0, 100, 8.846, 1.121])
#plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)
plt.axis("On")
cbar = plt.colorbar(im1)
cbar.set_label('Waveform amplitude', fontsize = 15, labelpad=25, rotation=270)
plt.clim(-5,5)

# plt.title('True Validation - Redatumed Validation', fontsize = 15, pad =12)
# plt.tick_params(axis='both', labelsize=12)
# plot_wiggle(tval_data-cval_data_3)
# plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
# plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)


fname = '66results.png'
plt.savefig(fname)
plt.show()

In [ ]:
#Evaluate norms
from numpy import linalg as LA
data1= cval_data_2
data1 = np.transpose(data1)
data2 = tval_data
data2 = np.transpose(data2)
num = data1-data2
denom = data2

num = LA.norm(num, 'fro')
denom = LA.norm(denom, 'fro')
data = num/denom
print(data)

In [ ]:
def wiggle_superimpose(data1, 
                       data2,
                      time,
                      title=None,
                      legend1 = 'Reference Model', 
                      legend2 = 'Perturbed Model',
                      ax=None,
                      skip=15,
                      perc=99.0,
                      gain=1.0,
                      rgb=(1, 2, 3),
                      alpha=1.0,
                      lw=0.5,
                      figsize=(16, 30),
                      predict=False,
                      save=True, 
                      fname= None):
        """
        Plots wiggle traces of seismic data.
        
        Args:
            data1 (ndarray): 2D seismic array. (reference model)
            data2 (ndarray): 2D seismic array. (perturbed model)
            time (ndarray): 1D seismic time array.
            ax (Axes): matplotlib Axes object (optional).
            skip (int): Skip=1, every trace, skip=2, every second trace, etc. #typically, skip = 15
            perc (float): Percentile to scale to, default 99%.
            gain (float): Gain value.
            rgb (tuple): 3-tuple of RGB for the trace colour.
            alpha (float): Opacity. Default 1.0.
            lw (float): Lineweigth. Default 0.5.
        Returns:
            Axes: A matplotlib Axes object.
        """
        if ax is None:
            fig = plt.figure(figsize=figsize) #default is (16,30)
            ax = fig.add_subplot(111)
        
        receiverPos = np.arange(0,100,skip)
        ntraces, nt = data1.shape
        rgba = list(rgb) + [alpha]
        sc1 = np.percentile(data1, perc)  # Normalization factor
        wigdata1 = data1[::skip, :]
        print (wigdata1.shape)
        sc2 = np.percentile(data2, perc)  # Normalization factor
        wigdata2 = data2[::skip, :]
        xpos = np.arange(ntraces)[::skip]
        
        i = 0

        for x, trace1, trace2 in zip(xpos, wigdata1, wigdata2):
            # Compute high resolution trace.
            amp1 = gain * trace1 / sc1 + x
            amp2 = gain * trace2 / sc2 + x
            t = 1000 * time
            hypertime = np.linspace(t[0], t[-1], (10 * t.size - 1) + 1)
            hyperamp1 = np.interp(hypertime, t, amp1)
            hyperamp2 = np.interp(hypertime, t, amp2)
            ax.tick_params(axis='both', labelsize=15)

            # Plot the line, then the fill.
            if i==0:
                ax.plot(hyperamp1, hypertime, 'k', lw=lw)
                ax.fill_betweenx(hypertime, hyperamp1, x,
                                 where=hyperamp1 > x,
                                 facecolor='limegreen',
                                 alpha=0.5,
                                 lw=0,
                                 label=legend1)
                print ('ture')
            else:
                ax.plot(hyperamp1, hypertime, 'k', lw=lw)
                ax.fill_betweenx(hypertime, hyperamp1, x,
                                 where=hyperamp1 > x,
                                 facecolor='limegreen',
                                 alpha=0.5,
                                 lw=0,
                                 )
            if i==0:  
                ax.plot(hyperamp2, hypertime, 'k', lw=lw)
                if predict: 
                    labelname = 'Predicted model'
                else:
                    labelname = legend2
                ax.fill_betweenx(hypertime, hyperamp2, x,
                                 where=hyperamp2 > x,
                                 facecolor='orange',
                                 alpha=0.5,
                                 lw=0,
                                 label=labelname)
            else:
                ax.plot(hyperamp2, hypertime, 'k', lw=lw)
                ax.fill_betweenx(hypertime, hyperamp2, x,
                                 where=hyperamp2 > x,
                                 facecolor='orange',
                                 alpha=0.5,
                                 lw=0,
                                )
            i+=1
        
        

        ax.invert_yaxis()
        l = plt.legend(prop={'size': 15})
        if title!= None:
            plt.title(title, fontsize = 15, pad =12)
        else:
            plt.title('Seismic Traces', fontsize = 15, pad =12)
        plt.xlabel('Receiver number', fontsize = 15, labelpad = 10)
        plt.xticks(receiverPos)
        plt.ylabel('Time (milliseconds)', fontsize = 12, labelpad = 10)
        plt.tight_layout()
        if save:
            plt.savefig(fname)
        return ax

In [ ]:
time = np.arange(0.001,6.601,0.012)
data1 = tval_data
data1 = np.transpose(data1)
data2 = pval_data_1
data2 = np.transpose(data2)
title_line = "Traces: True (Reference) Validation and Perturbed Data" 

#fname = 'MIT Images/SYMAE/Variable Seafloor/Trace_difference_0001.08.png'
plot = wiggle_superimpose(np.transpose(data1), np.transpose(data2), time, title=title_line, legend1='True Model', legend2='Perturbed Model', skip=5, figsize=(10,13), save=False, fname = fname)
#plot.figure

In [ ]:
time = np.arange(0.001,6.601,0.012)
data1 = tval_data
data1 = np.transpose(data1)
data2 = cval_data_1
data2 = np.transpose(data2)
title_line = "Traces: True (Reference) Validation and Corrected Validation Data" 

#fname = 'MIT Images/SYMAE/Variable Seafloor/Trace_difference_0001.08.png'
plot = wiggle_superimpose(np.transpose(data1), np.transpose(data2), time, title=title_line, legend1='True Model', legend2='Corrected Model', skip=5, figsize=(10,13), save=False, fname = fname)
#plot.figure

In [ ]:
time = np.arange(0.001,6.601,0.012)
data1 = pval_data_1
data1 = np.transpose(data1)
data2 = cval_data_1
data2 = np.transpose(data2)
title_line = "Traces: Perturbed Validation and Corrected Validation Data" 

#fname = 'MIT Images/SYMAE/Variable Seafloor/Trace_difference_0001.08.png'
plot = wiggle_superimpose(np.transpose(data1), np.transpose(data2), time, title=title_line, legend1='Perturbed Model', legend2='Corrected Model', skip=5, figsize=(10,13), save=False, fname = fname)
#plot.figure

In [ ]:
data1 = cval_data_1
data1 = np.transpose(data1)
data2 = tval_data
data2 = np.transpose(data2)
data = data1-data2
#------------------------------------------------------------------------------------------------------------------------
#Plot data
fig1 = plt.figure(figsize=(8,8))
plt.title('Perturbed - True, Model ' + str(901.08) +  '   (' + str(1.914) + '% pertubation)', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
im1 = plt.imshow((data), aspect='auto', cmap='RdBu', extent = [0, 100, 5896, 1250])
plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)
cbar = fig1.colorbar(im1)
cbar.set_label('Waveform amplitude', fontsize = 15, labelpad=25, rotation=270)
plt.clim(-5,5)
#fname = 'MIT Images/SYMAE/Variable Seafloor/Shot_gather_0001.00.png'
#plt.savefig(fname)
plt.show()

In [ ]:
from numpy import linalg as LA
data1= pval_data_3
data1 = np.transpose(data1)
data2 = tval_data
data2 = np.transpose(data2)
num = data1-data2
denom = data2

num = LA.norm(num, 'fro')
denom = LA.norm(denom, 'fro')
data = num/denom
print(data)
#------------------------------------------------------------------------------------------------------------------------
#Plot data
fig1 = plt.figure(figsize=(8,8))
plt.title('Perturbed - True, Model ' + str(901.08) +  '   (' + str(1.914) + '% pertubation)', fontsize = 15, pad =12)
plt.tick_params(axis='both', labelsize=12)
im1 = plt.imshow((data), aspect='auto', cmap='RdBu', extent = [0, 100, 5896, 1250])
plt.ylabel('Time (milliseconds)', fontsize = 15, labelpad = 10)
plt.xlabel("Receiver number", fontsize = 15, labelpad = 10)
cbar = fig1.colorbar(im1)
cbar.set_label('Waveform amplitude', fontsize = 15, labelpad=25, rotation=270)
plt.clim(-5,5)
#fname = 'MIT Images/SYMAE/Variable Seafloor/Shot_gather_0001.00.png'
#plt.savefig(fname)
plt.show()

In [ ]:
! cp /content/drive/MyDrive/seismicTL_circular1_Brindha/plots.py .
! pip install wiggle
from plots import *

def plot_seismic(X, labels, symencoder, nencoder, decoder):
    import numpy as np
    import matplotlib.pyplot as plt
    import os
    import random 
    import wiggle
    import tensorflow as tf
    from datetime import datetime
    from scipy import signal
    from sklearn.metrics import mean_squared_error
    now = datetime.now()
    # dir = now.strftime("%d-%m-%Y_%H")

    # if not os.path.exists(dir): os.mkdir(dir)

    nsamp=np.shape(X)[0]

    plt.figure(figsize=(40,50), facecolor='w')


    #i = np.random.randint(0, nsamp-1)
    # j=random.choice([ii for ii in range(nsamp) if labels[ii][str(0)]['medium']!=labels[i][str(0)]['medium']])
    # print("test_indices"+str(i)+","+str(j))

    i=0
    j=1

    Gi=symencoder.predict(X[i:i+1]) #symmetric latent predictions from model 0, for all instances
    Wi=nencoder.predict(X[i:i+1]) #nuisance latent predictions from model 0, for all instances

    Zii=np.concatenate((Gi,Wi),axis=1) #concatenate those two predictions, for all instances, in the latent space

    Xhatii=decoder.predict(Zii) #upsample with decoder from concatenated latent space

    num_instance_plot = 5    #how many perturbations we are gonna look at
    kki=random.choices(range(0,10),k=num_instance_plot) #pick 5 instances from the total instanecs -> ps: range excludes end index
    for ikk,kk in enumerate(kki):    #for each instance number 
        ID = labels[i][str(kk)]['ID']
        water_pert = round(labels_train[i][str(kk)]['water_pertubation'], 3)
        plt.subplot(6, num_instance_plot, ikk+1) #horizontal spreading of plots over 1st row.
        obs_data = X[i,kk,:,:,0]
        plot_wiggle(obs_data)               #plotting input test data of that perturbation
        plt.xticks([]); plt.yticks([]);
        plt.title('Observed; Model ' + str(ID) +  '   (' + str(water_pert) + '% pertubation)' )
        if ikk == 0:
            plt.ylabel('Exact')
                
        plt.subplot(6, num_instance_plot, num_instance_plot+ikk+1) #horizontal spreading of plots over 2d row
        prd_data = Xhatii[i,kk,:,:,0]
        plot_wiggle(prd_data)                        #plotting decoded test data of perturbation
        plt.xticks([]); plt.yticks([]);
        plt.title('Predicted; Model ' + str(ID) +  '   (' + str(water_pert) + '% pertubation)')
        if ikk == 0:
            plt.ylabel('Predict')

        plt.subplot(6, num_instance_plot, 2*num_instance_plot+ikk+1) #horizontal spreading of plots over 2d row
        residual = prd_data-obs_data
        plot_wiggle(residual)                        #plotting decoded test data of perturbation
        plt.xticks([]); plt.yticks([]);
        plt.title('Predicted-Observed; Model ' + str(ID) +  '   (' + str(water_pert) + '% pertubation)')
        if ikk == 0:
            plt.ylabel('Predict')

        
        
    # Gj=symencoder.predict(X[j:j+1])
    # Wj=nencoder.predict(X[j:j+1])
    # Zjj=np.concatenate((Gj,Wj),axis=1)
    # Zij=np.concatenate((Gi,Wj),axis=1)
    # Zji=np.concatenate((Gj,Wi),axis=1)
    # Xhatij=decoder.predict(Zij)
    # Xhatjj=decoder.predict(Zjj)
    # Xhatji=decoder.predict(Zji)
    # for ikk, kk in enumerate(kki):
    #     # style the jth bag with styles from the 0th bag
    #     plt.subplot(6, num_instance_plot, 3*num_instance_plot+ikk+1)
    #     a=Xhatji[0,kk,:,:,0]
    #     plot_wiggle(a,'r')
    #     plt.xticks([]); plt.yticks([]);

    #     plt.subplot(6, num_instance_plot, 4*num_instance_plot+ikk+1)
    #     b=X[j,kk,:,:,0]
    #     plot_wiggle(b)
    #     plt.xticks([]); plt.yticks([]);

    #     plt.subplot(6, num_instance_plot, 5*num_instance_plot+ikk+1)
    #     plot_wiggle(a-b)
    #     plt.xticks([]); plt.yticks([]);
    #     plt.title(str(mean_squared_error(a,b)))

    plt.savefig('../predictions.png',dpi=400)               
    plt.show()

In [ ]:
plot_seismic(Xo_test, labels_test,symencoder, nencoder, decoder)
#plt.savefig('/content/drive/MyDrive/seismicTL_circular1_Brindha/predictions.png',dpi=400) 

In [ ]:

labelindices={'base0': [], 'moni1' : [], 'moni2' : []}
for (i,label) in enumerate(labels):
    labelindices[label[0][0]].append(i)


# In[281]:


labels[labelindices['moni2']][0][0]


# # Visualize Latent Space

# In[ ]:



# In[80]:


# visualizing latent space

"""
fig=plt.figure(dpi=100, figsize=(15,8))
for label in ['100', '200', '300', '400', '500']:
    i = labelindices[label][0:1]
    
    with tf.device('/cpu:0'):
        latentsr=encoderr(Xo[i])
    
    

    """

with tf.device('/gpu'):
    test_large_indices = [np.random.randint(1, nsamp) for i in range(0,64)]
    latentsT=nencoder.predict(Xo_train                           [test_large_indices])
    latentsr=symencoder.predict(Xo_train[test_large_indices])
    #print(np.shape(latentsr))
    fig, axs = plt.subplots(2, 2, dpi=100, figsize=(15,8))
    a=axs[0,0].imshow(latentsr, aspect=1)
    #for l in latentsr:
    #    a=axs[0,0].plot(l)
    axs[0,0].set_title('G')
    plt.colorbar(a, ax=axs[0,0]) 

    #a=axs[1,0].imshow(tf.exp(latentsr[1]), aspect=1)
    #axs[1,0].set_title('sigma encoderr output for samples')
    #plt.colorbar(a, ax=axs[1,0]) 

    b=axs[0,1].imshow(latentsT, aspect=1)
    axs[0,1].set_title('Z')
    plt.colorbar(b, ax=axs[0,1]) 
    
plt.gcf()
#b=axs[1,1].imshow(tf.exp(latentsT[1]), aspect=1)
#axs[1,1].set_title('sigma encoderT output for samples')
#plt.colorbar(b, ax=axs[1,1]) 


# In[67]:


plt.imshow(latentsT)


# In[68]:


with tf.device('/gpu'):
    latentsr={'base0': [], 'moni1':[],'moni2':[]}
    for label in ['base0', 'moni1', 'moni2']:
        i = labelindices[label][1:20]
        with tf.device('/gpu:0'):
            latentsr[label]=symencoder.predict(Xo_train[i])


# In[69]:


fig, ax = plt.subplots(2,5,dpi=500, figsize=(3,2))
markers=['r', 'b', 'c', 'g', 'm']
subs=['(a)', '(b)', '(c)', '(d)', '(e)']
for (i,label) in enumerate(['base0', 'moni1', 'moni2', 'base0', 'base0']):
    lmean=np.mean(latentsr[label], axis=0)
    #ax[i].plot(lmean, color='k', linewidth=0.1)
    ax[1,i].imshow(latentsr[label].T)
    # Hide grid lines
    ax[1,i].grid(False)
#     Hide axes ticks
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])
    ax[1,i].set_xlabel('$\tau$')
    ax[1,i].set_ylabel('$Z_g$')
    
    
    
    ax[0,i].imshow(vp[label].T, cmap='RdBu')
    ax[0,i].axis('off')
    ax[0,i].set_xlabel('$x$')
    ax[0,i].set_ylabel('$z$')
    ax[0,i].set_title(subs[i]+'$\epsilon=$'+str(i))


    #for l in latentsr[label]:
     #   ax[i].fill_between(range(0,96), l, lmean, color=markers[i], alpha=0.1)
        #ax[i].set_xlabel()
        
plt.gcf()



In [ ]:
#TO tRAIN 

batch_size=2
ds_train=tf.data.Dataset.from_tensor_slices((Xo_train, Xo_train))
ds_train = ds_train.shuffle(nsamp,reshuffle_each_iteration=True) 
ds_train=ds_train.batch(batch_size)
ds_train = ds_train.cache()
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test=tf.data.Dataset.from_tensor_slices((Xo_test, Xo_test))
ds_test=ds_test.batch(batch_size)
history=model.fit(ds_train,epochs=50,  shuffle=True, validation_data=ds_test,)